In [1]:
#import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from citipy import citipy
#import the requests library
import requests

#import the api key
from config import weather_api_key

In [2]:
#create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
#add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

739

In [5]:
#starting url
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [15]:
# create an empty list to hold the weather data
city_data = []
#print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

#create counter
record_count = 1
set_count = 1

#loop through all the cities in the list
for i, city in enumerate(cities):
    
    #group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    #create endpoin url with each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    #log the url, record, and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    #add 1 to the record count
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_description = city_weather["weather"][0]["description"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | yulara
Processing Record 2 of Set 1 | jamestown
Processing Record 3 of Set 1 | kapaa
Processing Record 4 of Set 1 | bonito
Processing Record 5 of Set 1 | ribeira grande
Processing Record 6 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 7 of Set 1 | padang
Processing Record 8 of Set 1 | albany
Processing Record 9 of Set 1 | zhigansk
Processing Record 10 of Set 1 | ushuaia
Processing Record 11 of Set 1 | port alfred
Processing Record 12 of Set 1 | constitucion
Processing Record 13 of Set 1 | yellowknife
Processing Record 14 of Set 1 | georgetown
Processing Record 15 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 16 of Set 1 | pevek
Processing Record 17 of Set 1 | rikitea
Processing Record 18 of Set 1 | tanout
Processing Record 19 of Set 1 | saskylakh
Processing Record 20 of Set 1 | los llanos de aridane
Processing Record 21 of Set 1 | tuatapere
Pro

In [16]:
#convert the array of dictionaries to a pandas dataframe
city_data_df = pd.DataFrame(city_data)
#change the order of the columns
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]

city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Yulara,AU,-25.2406,130.9889,62.60,55,1,5.75,clear sky
1,Jamestown,US,42.0970,-79.2353,66.20,45,20,3.44,few clouds
2,Kapaa,US,22.0752,-159.3190,75.20,57,75,23.02,broken clouds
3,Bonito,BR,-21.1211,-56.4819,79.61,50,92,5.77,overcast clouds
4,Ribeira Grande,PT,38.5167,-28.7000,60.48,86,13,8.86,few clouds
5,Padang,ID,-0.9492,100.3543,75.20,94,20,2.30,moderate rain
6,Albany,US,42.6001,-73.9662,63.00,23,20,8.05,few clouds
7,Zhigansk,RU,66.7697,123.3711,-7.55,99,97,11.99,overcast clouds
8,Ushuaia,AR,-54.8000,-68.3000,51.80,66,75,6.26,light rain
9,Port Alfred,ZA,-33.5906,26.8910,63.95,86,57,13.53,broken clouds


In [17]:
#create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
#export the city_data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")